In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 이 데이터는 가상데이터 5개를 합친 sum의 전력량과 조도 데이터입니다.
# " 이 파일 경로는 제 구글 드라이브 기준입니다. 사용하시는 분의 경로에 맞게 바꾸셔야합니다
df = pd.read_excel("/content/drive/MyDrive/bigpj/gs/sum.xlsx")

In [4]:
df

,시리얼,전력량1 (Wh),조도1 (%),등록일시,구분
0,KCN-1007,0,100,2022-10-24 01:03:30,일반군
1,KCN-1007,0,100,2022-10-24 02:04:17,일반군
2,KCN-1007,0,100,2022-10-24 03:06:35,일반군
3,KCN-1007,0,100,2022-10-24 04:07:15,일반군
4,KCN-1007,0,100,2022-10-24 05:03:40,일반군
...,...,...,...,...,...
831,KCN-1004,87,94,2023-06-07 19:00:00,고위험군
832,KCN-1004,15,32,2023-06-07 20:00:00,고위험군
833,KCN-1004,50,48,2023-06-07 21:00:00,고위험군
834,KCN-1004,57,19,2023-06-07 22:00:00,고위험군


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   시리얼        836 non-null    object        
 1   전력량1 (Wh)  836 non-null    int64         
 2   조도1 (%)    836 non-null    int64         
 3   등록일시       836 non-null    datetime64[ns]
 4   구분         836 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 32.8+ KB


In [6]:
df.isna().sum()

시리얼          0
전력량1 (Wh)    0
조도1 (%)      0
등록일시         0
구분           0
dtype: int64

In [7]:
df['시리얼']

0      KCN-1007
1      KCN-1007
2      KCN-1007
3      KCN-1007
4      KCN-1007
         ...   
831    KCN-1004
832    KCN-1004
833    KCN-1004
834    KCN-1004
835    KCN-1004
Name: 시리얼, Length: 836, dtype: object

In [8]:
import numpy as np

def detect_no_power_and_brightness_variation(df, detection_periods):
    unique_serials = df['시리얼'].unique()

    for serial in unique_serials:
        df_serial = df[df['시리얼'] == serial]
        danger_signals = []

        anomaly_detected = False  # Flag to track if anomaly signal has been detected

        for detection_period in detection_periods:
            for i in range(len(df_serial) - detection_period + 1):
                power_usage = df_serial['전력량1 (Wh)'].iloc[i:i + detection_period]
                brightness = df_serial['조도1 (%)'].iloc[i:i + detection_period]
                classification = df_serial['구분'].iloc[i:i + detection_period]

                # Check if the classification group is 'high-risk'
                if '고위험군' in classification.values:
                    # Check if both power usage and brightness have no fluctuation (constant values)
                    if np.std(power_usage) == 0 and np.std(brightness) == 0:
                        start_time = df_serial['등록일시'].iloc[i]
                        end_time = df_serial['등록일시'].iloc[i + detection_period - 1]
                        danger_signals.append((start_time, end_time, "고위험군 이상 신호가 감지되었습니다"))

                # Check if the classification group is 'risk'
                elif '위험군' in classification.values:
                    # Perform checks specific to the risk group
                    if np.std(power_usage) == 0 and np.std(brightness) == 0:
                        start_time = df_serial['등록일시'].iloc[i]
                        end_time = df_serial['등록일시'].iloc[i + detection_period - 1]
                        danger_signals.append((start_time, end_time, "위험군 이상 신호가 감지되었습니다"))

                # Check if the classification group is 'normal group'
                elif '일반군' in classification.values:
                    # Perform checks specific to the normal group
                    if np.std(power_usage) == 0 and np.std(brightness) == 0 and detection_period == 50:
                        start_time = df_serial['등록일시'].iloc[i]
                        end_time = df_serial['등록일시'].iloc[i + detection_period - 1]
                        danger_signals.append((start_time, end_time, "일반군 이상 신호가 감지되었습니다."))

            if len(danger_signals) > 0:
                if '고위험군' in classification.values:
                    print(f"고위험군 이상 신호가 감지되었습니다. (serial: {serial}):")
                elif '위험군' in classification.values:
                    print(f"위험군 이상 신호가 감지되었습니다. (serial: {serial}):")
                elif '일반군' in classification.values:
                    print(f"일반군 이상 신호가 감지되었습니다. (serial: {serial}):")

                for signal in danger_signals:
                    print(f"{signal[0]} to {signal[1]} - {signal[2]}")
                break

        if len(danger_signals) == 0 and not anomaly_detected:
            print(f"이상 신호가 감지되지 않았습니다 (serial: {serial})")
        elif len(danger_signals) > 0 and not anomaly_detected:
            print("이상 신호가 감지되었습니다.")

# Assuming you have a DataFrame called 'df' containing the relevant data

# Define the detection periods
detection_periods = [24, 36, 50]

# Call the function
detect_no_power_and_brightness_variation(df, detection_periods)

이상 신호가 감지되지 않았습니다 (serial: KCN-1007)
이상 신호가 감지되지 않았습니다 (serial: KCN-1116)
일반군 이상 신호가 감지되었습니다. (serial: KCN-0730):
2023-06-05 00:00:00 to 2023-06-07 01:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 01:00:00 to 2023-06-07 02:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 02:00:00 to 2023-06-07 03:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 03:00:00 to 2023-06-07 04:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 04:00:00 to 2023-06-07 05:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 05:00:00 to 2023-06-07 06:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 06:00:00 to 2023-06-07 07:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 07:00:00 to 2023-06-07 08:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 08:00:00 to 2023-06-07 09:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 09:00:00 to 2023-06-07 10:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 10:00:00 to 2023-06-07 11:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 11:00:00 to 2023-06-07 12:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 12:00:00 to 2023-06-07 13:00:00 - 일반군 이상 신호가 감지되었습니다.
2023-06-05 13:00:00 to 2023-06-07 14:00:0

In [ ]:
# 데이터셋 제작 코드입니다. 신경안써도 됩니다.
import pandas as pd
import numpy as np

# Define the sleep time period (from 23:00 to 7:00)
sleep_start = 23
sleep_end = 7

# Define the fixed values for 50 hours
fixed_power_consumption = 30
fixed_illuminance = 80

# Number of weeks for the data
num_weeks = 1

# Number of rows and columns for each week of data
num_rows_per_week = 7 * 24  # 7 days * 24 hours
num_columns = 2  # Power Consumption, Illuminance

# Create a list of timestamps for the hourly records
timestamps = pd.date_range(start='2023-06-01', periods=num_rows_per_week * num_weeks, freq='H')

# Create empty arrays for power consumption and illuminance
power_consumption = np.zeros(num_rows_per_week * num_weeks, dtype=int)
illuminance = np.zeros(num_rows_per_week * num_weeks, dtype=int)

# Generate the dataset
for i, timestamp in enumerate(timestamps):
    hour = timestamp.hour

    if i % (24 * 7) in range(24 * 3, 24 * 3 + 50):
        # Fixed value period
        power_consumption[i:i+50] = fixed_power_consumption
        illuminance[i:i+50] = fixed_illuminance
    elif sleep_start <= hour or hour < sleep_end:
        # Sleep time
        power_consumption[i] = 0
        illuminance[i] = 0
    else:
        # Awake time
        power_consumption[i] = np.random.randint(1, 101)  # Random power consumption between 1 and 100
        illuminance[i] = np.random.randint(1, 101)  # Random illuminance between 1 and 100

# Create the DataFrame
df = pd.DataFrame({'Timestamp': timestamps, 'Power Consumption': power_consumption, 'Illuminance': illuminance})

# Save the DataFrame as a CSV file
df.to_csv('data.csv', index=False)
